# ⏱️ Real-Time PYUSD Tracker (GCP Node Engine)
Monitors PYUSD Transfers on Ethereum in near real-time using GCP's Blockchain Node Engine.
Exports latest transfers to Google Sheets.

In [ ]:
# 📦 Install dependencies
!pip install --quiet gspread gspread_dataframe requests web3 google-auth

In [ ]:
# 📚 Imports and Auth
import time
import json
import requests
import pandas as pd
from web3 import Web3
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth

# Authenticate with Google Cloud
auth.authenticate_user()
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

print("✅ Google Authenticated")

In [ ]:
# 🔌 Connect to GCP Blockchain Node Engine (Ethereum Mainnet)
GCP_RPC_URL = 'https://<your-node>.blockchainnodeengine.googleapis.com'  # Replace with your actual URL
web3 = Web3(Web3.HTTPProvider(GCP_RPC_URL))

PYUSD_CONTRACT = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'.lower()
TRANSFER_TOPIC = web3.keccak(text="Transfer(address,address,uint256)").hex()

print("🔗 Connected to Ethereum via GCP RPC")

In [ ]:
# 📊 Setup Google Sheet for real-time updates
sheet = gc.create("PYUSD_Realtime_Transfers")
ws = sheet.get_worksheet(0)
print("📤 Output Sheet:", sheet.url)

In [ ]:
# ⏱️ Real-time polling loop
latest_block = web3.eth.block_number
poll_interval = 15  # seconds

while True:
    new_block = web3.eth.block_number
    if new_block > latest_block:
        print(f"🔍 Scanning block {new_block}")
        logs = web3.eth.get_logs({
            'fromBlock': new_block,
            'toBlock': new_block,
            'address': PYUSD_CONTRACT,
            'topics': [TRANSFER_TOPIC]
        })

        transfers = []
        for log in logs:
            try:
                from_addr = '0x' + log['topics'][1].hex()[-40:]
                to_addr = '0x' + log['topics'][2].hex()[-40:]
                value = int(log['data'], 16) / 1e18
                transfers.append({
                    'block': log['blockNumber'],
                    'tx_hash': log['transactionHash'].hex(),
                    'from': from_addr,
                    'to': to_addr,
                    'value': value
                })
            except Exception as e:
                print("⚠️ Error parsing log:", e)

        if transfers:
            df = pd.DataFrame(transfers)
            set_with_dataframe(ws, df)
            print(f"✅ {len(df)} transfer(s) exported to Google Sheets")
        else:
            print("ℹ️ No PYUSD transfers in this block")

        latest_block = new_block

    time.sleep(poll_interval)